In [67]:
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'.
There are 4 directories and 0 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\test'.
There are 0 directories and 742 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\test\Cyst'.
There are 0 directories and 1016 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\test\Normal'.
There are 0 directories and 276 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\test\Stone'.
There are 0 directories and 457 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\test\Tumor'.
There are 4 directories and 0 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\train'.
There are 0 directories and 2967 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\train\Cyst'.
There are 0 directories and 4061 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\train\Normal'.
There are 0 directories and 1101 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\train\Stone'.
There are 0 directories and 1826 images i

In [43]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/train/"
test_dir = "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)


In [44]:
print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("Testing images:")
test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")
     

Training images:
Found 9955 images belonging to 4 classes.
Testing images:
Found 2491 images belonging to 4 classes.


In [45]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [46]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

# Define IMAGE_SHAPE based on the expected input shape of the model
IMAGE_SHAPE = (224, 224)

def create_model(model_url, num_classes=4):
    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                             trainable=False,  # freeze the underlying patterns
                                             name='feature_extraction_layer',
                                             input_shape=IMAGE_SHAPE + (3,))  # define the input image shape

    # Create our own model
    model = tf.keras.Sequential([
        feature_extractor_layer,  # use the feature extraction layer as the base
        layers.Dense(num_classes, activation='softmax', name='output_layer')  # create our own output layer
    ])

    return model

# Example usage
model1 = create_model("https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-classification/versions/4", num_classes=4)


In [47]:
model1.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [48]:
resnet_history = model1.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data))

Epoch 1/5
312/312 [==============================] - 385s 1s/step - loss: 0.5252 - accuracy: 0.8097 - val_loss: 0.3248 - val_accuracy: 0.8940
Epoch 2/5
312/312 [==============================] - 221s 707ms/step - loss: 0.2566 - accuracy: 0.9189 - val_loss: 0.2509 - val_accuracy: 0.9057
Epoch 3/5
312/312 [==============================] - 218s 699ms/step - loss: 0.1906 - accuracy: 0.9388 - val_loss: 0.2001 - val_accuracy: 0.9382
Epoch 4/5
312/312 [==============================] - 218s 698ms/step - loss: 0.1561 - accuracy: 0.9499 - val_loss: 0.1669 - val_accuracy: 0.9478
Epoch 5/5
312/312 [==============================] - 220s 706ms/step - loss: 0.1346 - accuracy: 0.9588 - val_loss: 0.1589 - val_accuracy: 0.9522


In [49]:
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 1001)              3540265   
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 4)                 4008      
                                                                 
Total params: 3544273 (13.52 MB)
Trainable params: 4008 (15.66 KB)
Non-trainable params: 3540265 (13.51 MB)
_________________________________________________________________


In [56]:
# Add these lines before the fit function
for data, labels in train_data_10_percent:
    print("Data shape:", data.shape)
    print("Labels shape:", labels.shape)
    break  # Print the shape of the first batch only


Data shape: (32, 224, 224, 3)
Labels shape: (32, 4)


# Setup MLflow in Terminal:
# Command : 
 mlflow ui

In [58]:
import mlflow
experiment_name = "MLops_Production"
mlflow.set_experiment(experiment_name)

2024/01/12 15:05:37 INFO mlflow.tracking.fluent: Experiment with name 'MLops_Production' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/361568761679039183', creation_time=1705052137993, experiment_id='361568761679039183', last_update_time=1705052137993, lifecycle_stage='active', name='MLops_Production', tags={}>

In [59]:
import mlflow
from mlflow.tensorflow import MLflowCallback

mlflow.set_tracking_uri('http://127.0.0.1:5000')

# Create and compile your model

# Start the MLflow run
with mlflow.start_run() as run:

    # Log the model parameters
    mlflow.log_param("epochs", 5)
    mlflow.log_param("steps_per_epoch", len(train_data_10_percent))
    # Log other hyperparameters as needed

    # Train your model with MLflowCallback
    resnet_history = model1.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  callbacks=[MLflowCallback(run)]
                                  )

    # Log the final metrics after training
    mlflow.log_metrics({
        "final_loss": resnet_history.history["loss"][-1],
        "final_accuracy": resnet_history.history["accuracy"][-1],
        "final_val_loss": resnet_history.history["val_loss"][-1],
        "final_val_accuracy": resnet_history.history["val_accuracy"][-1],
    })
    
    mlflow.tensorflow.log_model(model1, "model")

    mlflow_run_id = run.info.run_id


Epoch 1/5
312/312 [==============================] - 225s 721ms/step - loss: 0.1164 - accuracy: 0.9647 - val_loss: 0.1543 - val_accuracy: 0.9470
Epoch 2/5
312/312 [==============================] - 218s 697ms/step - loss: 0.1032 - accuracy: 0.9695 - val_loss: 0.1277 - val_accuracy: 0.9582
Epoch 3/5
312/312 [==============================] - 221s 708ms/step - loss: 0.0919 - accuracy: 0.9744 - val_loss: 0.1469 - val_accuracy: 0.9502
Epoch 4/5
312/312 [==============================] - 215s 690ms/step - loss: 0.0848 - accuracy: 0.9770 - val_loss: 0.1124 - val_accuracy: 0.9631
Epoch 5/5
312/312 [==============================] - 215s 688ms/step - loss: 0.0768 - accuracy: 0.9796 - val_loss: 0.1075 - val_accuracy: 0.9627


2024/01/12 15:23:55 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpmfvphbqh\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpmfvphbqh\model\data\model\assets


In [60]:
mlflow.tensorflow.save_model(model1, "model_final_2.0")

2024/01/12 15:25:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: e:\datascience\model_final_2.0\data\model\assets


INFO:tensorflow:Assets written to: e:\datascience\model_final_2.0\data\model\assets


In [61]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow_hub as hub


model2 = mlflow.tensorflow.load_model("model_final_2.0")


# Define IMAGE_SHAPE based on the expected input shape of the model
IMAGE_SHAPE = (224, 224)

# Define class labels
class_labels = ['Cyst', 'Normal', 'Stone', 'Tumor']

# Function to preprocess an image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMAGE_SHAPE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  
    return img_array

def predict_class(image_path):
    img_array = preprocess_image(image_path)
    predictions = model2.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

# Example image path
image_path = 'Normal- (705).jpg'

# Make a prediction
predicted_label = predict_class(image_path)

# Print the predicted label
print("Predicted label:", predicted_label)


1/1 [==============================] - 1s 1s/step
Predicted label: Normal


In [62]:
model_name = "Kidney_CT_Scan"
model_version = 1

In [63]:
print("MLFlow Run ID: ",mlflow_run_id)
logged_model_path = f"runs:/{mlflow_run_id}/model"

MLFlow Run ID:  0831f5273e0a466d8d72f100734db7da


In [64]:
with mlflow.start_run(run_id=mlflow_run_id) as run:
    result = mlflow.register_model(
        logged_model_path,
        model_name
    )

Successfully registered model 'Kidney_CT_Scan'.
2024/01/12 15:26:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Kidney_CT_Scan, version 1
Created version '1' of model 'Kidney_CT_Scan'.


In [65]:
client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Production"
)


C:\Users\HP\AppData\Local\Temp\ipykernel_31316\128635073.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1705053378906, current_stage='Production', description='', last_updated_timestamp=1705053382831, name='Kidney_CT_Scan', run_id='0831f5273e0a466d8d72f100734db7da', run_link='', source='mlflow-artifacts:/361568761679039183/0831f5273e0a466d8d72f100734db7da/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow_hub as hub


model2 = mlflow.tensorflow.load_model("model_final")


# Define IMAGE_SHAPE based on the expected input shape of the model
IMAGE_SHAPE = (224, 224)

# Define class labels
class_labels = ['Cyst', 'Normal', 'Stone', 'Tumor']

# Function to preprocess an image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMAGE_SHAPE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  
    return img_array

def predict_class(image_path):
    img_array = preprocess_image(image_path)
    predictions = model2.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

# Example image path
image_path = 'Normal- (705).jpg'

# Make a prediction
predicted_label = predict_class(image_path)

# Print the predicted label
print("Predicted label:", predicted_label)


1/1 [==============================] - 1s 880ms/step
Predicted label: Normal


In [ ]:
loaded_model = mlflow.tensorflow.load_model(
    model_uri=f"models:/{model_name}/production"
)
